# 1 Initial setup

Import modules

In [ ]:
import csv
import sys
import datetime
import copy

Set variables for this run

In [ ]:
namespaceUri = 'http://rs.tdwg.org/ac/terms/'
database = 'audubon'
vocabulary = 'ac'
namespace = 'terms'
date_issued = '2019-09-30'
local_offset_from_utc = '-05:00'
versions = database + '-versions'
modifications_filename = 'mods.csv'
version_namespace = namespaceUri + 'version/'

Define utility functions

In [ ]:
def readCsv(filename):
    fileObject = open(filename, 'r', newline='', encoding='utf-8')
    readerObject = csv.reader(fileObject)
    array = []
    for row in readerObject:
        array.append(row)
    fileObject.close()
    return array

def writeCsv(fileName, array):
    fileObject = open(fileName, 'w', newline='', encoding='utf-8')
    writerObject = csv.writer(fileObject)
    for row in array:
        writerObject.writerow(row)
    fileObject.close()

    # returns a list with first item Boolean and second item the index
def findColumnWithHeader(header_row_list, header_label):
    found = False
    for column_number in range(0, len(header_row_list)):
        if header_row_list[column_number] == header_label:
            found = True
            found_column = column_number
    if found:
        return [True, found_column]
    else:
        return [False, 0]
    
def isoTime(offset):
    currentTime = datetime.datetime.now()
    return currentTime.strftime("%Y-%m-%dT%H:%M:%S") + offset

# 2 Extract information from metadata files

**Notes for all tables:** row 0 (the first row) is a header row. The columns whose headers contain `replaces1_` and `replaces2_` are used in the uncommon situation where a term or version replaces more than one other term or version.  In such cases, the tables would need to be modified manually.  

The table containing the **modifications** (term additions and changes) looks like this:

![](mods-table.png)

The `term_localName` column is the primary key for this table.

The table containing **current terms metadata** looks like this:

![](current-terms-table1.png)
![](current-terms-table2.png)

Notice that all of the column headers from `label` onwards to the right are the same as those in the modifications table. The column headers to the left of `label` are ideosyncratic for the current terms table type and must be handled specially.  The `term_localName` column is the primary key for this table.  The `term_deprecated` and `replaces_term` columns are for unusual situations and aren't managed by this script.  They would need to be edited manually when those situations occur.

The table containing **versions metadata** looks like this:

![](versions-table1.png)
![](versions-table2.png)

As with the current terms table, all of the column headers from `label` onwards to the right are the same as the previous two tables. The column headers to the left of `label` are ideosyncratic for the table type and must be handled specially. The `versionLocalName` column is the primary key for this table.  The `term_localName` column is a foreign key that relates rows in this table to rows in the other two tables.  

## 2.1 Read in the tables of current terms and of modifications

In [ ]:
terms_metadata_filename = database + '/' + database + '.csv'
terms_metadata = readCsv(terms_metadata_filename)

modifications_metadata = readCsv(modifications_filename)
print('Current terms table headers: ', terms_metadata[0])
print()
print('Modifications table headers: ', modifications_metadata[0])

The script loads data only on the basis of the column names and not their position.  So a number of variables are defined that hold the column numbers for various fields.  

Find which column numbers in the modifications file and the metadata file hold the term local name.  This column is the primary key for the table and therefore is used to match rows for terms that need to be modified with the corresponding rows in the current term metadata table.  The `term_localName` column is the only one in the modifications table that is not potentially a vocabulary-specific metadata field.

In [ ]:
result = findColumnWithHeader(modifications_metadata[0], 'term_localName')
if result[0] == False:
    print('The modifications file does not have a term_localName column')
    sys.exit
else:
    mods_local_name = result[1]

# don't error trap here because all existing files should have a local name column header
result = findColumnWithHeader(terms_metadata[0], 'term_localName')
metadata_localname_column = result[1]
print('Modifications table local name column: ', mods_local_name)
print('Term metadata table local name column: ', metadata_localname_column)

Create a list of the local names of terms to be added or modified

In [ ]:
mods_term_localName = []
for term_number in range(1, len(modifications_metadata)):
    mods_term_localName.append(modifications_metadata[term_number][mods_local_name])
print(mods_term_localName)

Find out which terms are new terms and which are modified old terms.  Create a list for each.

In [ ]:
new_terms = []
modified_terms = []
for test_term in mods_term_localName:
    found = False
    for term in terms_metadata:
        if test_term == term[metadata_localname_column]:
            found = True
            modified_terms.append(test_term)
    if not found:
        new_terms.append(test_term)
print('New terms: ', new_terms)
print('Modified terms: ', modified_terms)

# 3 Update versions-related metadata

The master versions metadata table (in the `xx-versions` directory) is used to generate metadata about versions.  The versions join table (in the current terms directory) is a minimal joins table used to generate the `hasVersion` links for current terms.  The versions replacements table is used to generate `dcterms:replaces` and `dcterms:isReplacedBy` links for version metadata records.

## 3.1 Load master versions metadata and determine positions of special columns

Read in the term versions metadata file

In [ ]:
term_versions_metadata_filename = versions + '/' + versions + '.csv'
term_versions_metadata = readCsv(term_versions_metadata_filename)
print('Version headers: ', term_versions_metadata[0])

Find the positions of the ideosyncratic version columns.  Recall:

![](versions-table1.png)
![](versions-table2.png)

In [ ]:
version_modified = findColumnWithHeader(term_versions_metadata[0], 'document_modified')[1]
version_column = findColumnWithHeader(term_versions_metadata[0], 'version')[1]
version_local_name = findColumnWithHeader(term_versions_metadata[0], 'versionLocalName')[1]
version_isDefinedBy = findColumnWithHeader(term_versions_metadata[0], 'version_isDefinedBy')[1]
version_issued = findColumnWithHeader(term_versions_metadata[0], 'version_issued')[1]
version_status = findColumnWithHeader(term_versions_metadata[0], 'version_status')[1]
replaces_version = findColumnWithHeader(term_versions_metadata[0], 'replaces_version')[1]
version_term_local_name_column = findColumnWithHeader(term_versions_metadata[0], 'term_localName')[1]

## 3.2 Supersede old versions of the modified terms

Go through each version and supersede any that match the local names of the modified terms

In [ ]:
print('rows in versions table to be superseded:')
for term in modified_terms:
    for version_row in range(1, len(term_versions_metadata)):
        if term_versions_metadata[version_row][version_term_local_name_column] == term:
            print(version_row, term)
            term_versions_metadata[version_row][version_status] = 'superseded'
            term_versions_metadata[version_row][version_modified] = isoTime(local_offset_from_utc)

## 3.3 Create new versions of new and modified terms

Make sure that all columns in modified terms file are in the term versions file

In [ ]:
for column in modifications_metadata[0]:
    result = findColumnWithHeader(term_versions_metadata[0], column)
    if result[0] == False:
        print('The modifications file is missing the ', column, ' column.')
        sys.exit

Open the versions join table, which looks like this:

![](version-joins-table.png)

In [ ]:
versions_join_table_filename = database + '/' + versions + '.csv'
versions_join_table = readCsv(versions_join_table_filename)
print('Versions join table headers: ', versions_join_table[0])

Create new versions and new version joins lists

In [ ]:
newVersions = []
newVersionJoins = []

Create a row in the new term versions list for the added or modified terms

In [ ]:
for row_number in range(1, len(modifications_metadata)):
    newVersion = []
    # create a column for every column in the term version file
    for column in term_versions_metadata[0]:
        # find the column in the modifications file that matches the version column and add its value
        result = findColumnWithHeader(modifications_metadata[0], column)
        if result[0] == True:
            newVersion.append(modifications_metadata[row_number][result[1]])
        else:
            newVersion.append('')
    # set the modification dateTime for the newly created version
    newVersion[version_modified] = isoTime(local_offset_from_utc)
    newVersions.append(newVersion)

Insert metadata specific to the new versions

In [ ]:
for rowNumber in range(0, len(newVersions)):
    # need to add one to the row of modifications_metadata because it includes a header row
    currentTermLocalName = modifications_metadata[rowNumber + 1][mods_local_name]
    newVersions[rowNumber][version_issued] = date_issued
    newVersions[rowNumber][version_status] = 'recommended'
    newVersions[rowNumber][version_local_name] = currentTermLocalName + '-' + date_issued
    newVersions[rowNumber][version_isDefinedBy] = version_namespace
    newVersions[rowNumber][version_column] = version_namespace + currentTermLocalName + '-' + date_issued

    # if the new version replaces an older one for the term, we need to provide a value for the `replaces_version` column
    if currentTermLocalName in modified_terms:
        # look through metadata for old versions to find the most recent version of the term
        mostRecent = 'a' # start with a string value earlier in alphabetization than any term version URI
        for version_row in range(1, len(term_versions_metadata)):
            if term_versions_metadata[version_row][version_term_local_name_column] == currentTermLocalName:
                # Make it the mostRecent if it's later than the previous mostRecent
                if term_versions_metadata[version_row][version_column] > mostRecent:
                    mostRecent = term_versions_metadata[version_row][version_column]
        # insert the most recent version found into the appropriate column
        newVersions[rowNumber][replaces_version] = mostRecent
    
    # create a join record for each new version and add it to the list of new joins
    newVersionJoin =[ newVersions[rowNumber][version_column], modifications_metadata[rowNumber + 1][mods_local_name] ]
    newVersionJoins.append(newVersionJoin)

print(newVersions)
print()
print(newVersionJoins)

Append the new versions to the old version tables and save the file

In [ ]:
revised_term_versions_metadata = term_versions_metadata + newVersions
writeCsv('temp_versions.csv', revised_term_versions_metadata)

revised_term_versions_joins = versions_join_table + newVersionJoins
writeCsv('temp_version_joins.csv', revised_term_versions_joins)

## 3.4 Update the versions replacements table

The versions replacements table links a newly created version of a modified term to the previous version it is replacing. 

Since new terms do not have any previous versions, they will not have entries in this table. Thus replacement entries only need to be generated for terms on the `modified_terms` list.

The script only handles cases where there is a new version of a term with the same local name.  In cases where an old term is deprecated and is replaced by a new term with a different local name, the replacement table will need to be updated manually.

Here's an example of what a replacements table looks like:

![](version-replacements-table.png)

Open it:

In [ ]:
versions_replacements_table_filename = versions + '/' + versions + '-replacements.csv'
versions_replacements_table = readCsv(versions_replacements_table_filename)
print('Versions replacements table headers: ', versions_replacements_table[0])

Find the most recent previous version for each term on the `modified_terms` list and generate a replacement record for it.

In [ ]:
# create a list to hold the newly generated replacements rows
newReplacements = []

for modifiedTerm in modified_terms:
    # generate the newly created version local name for the modified term
    newVersion = modifiedTerm  + '-' + date_issued
    # step through the list of previous versions and find the one with the most recent issued date
    mostRecent = 'a'
    for oldVersion in versions_join_table:
        # the second column in the join table is the term local name
        if oldVersion[1] == modifiedTerm:
            # the first column in the join table is the full version URI. Make it the mostRecent if it's later than the previous mostRecent
            if oldVersion[0] > mostRecent:
                mostRecent = oldVersion[0]
    newReplacements.append([mostRecent, newVersion])
print(newReplacements)

Append the new replacement list to the existing table and save as a file

In [ ]:
revised_versions_replacements_table = versions_replacements_table + newReplacements
writeCsv('temp_replacements.csv', revised_versions_replacements_table)

# 4 Process current terms metadata

## 4.1 determine positions of special columns

Find the positions of the ideosyncratic current terms columns (table already loaded). Recall:

![](current-terms-table1.png)
![](current-terms-table2.png)

In [ ]:
term_modified_dateTime = findColumnWithHeader(terms_metadata[0], 'document_modified')[1]
term_localName = findColumnWithHeader(terms_metadata[0], 'term_localName')[1]
term_modified = findColumnWithHeader(terms_metadata[0], 'term_modified')[1]
term_created = findColumnWithHeader(terms_metadata[0], 'term_created')[1]
term_isDefinedBy = findColumnWithHeader(terms_metadata[0], 'term_isDefinedBy')[1]

## 4.2 Modify current terms metadata table

Each item in the term modifications list will either modify existing term metadata or add new term metadata

In [ ]:
print('Changed current terms rows: ')
# step through each row in the modification metadata table and modify existing current terms when applicable
for mods_rownumber in range(1, len(modifications_metadata)):
    mods_localname_string = modifications_metadata[mods_rownumber][mods_local_name]
    modified = False
    for term_name in modified_terms:
        # only make a modification if it's on the list of terms to be modified
        if mods_localname_string == term_name:
            modified = True
    # this section of code modifies existing terms
    if modified:
        # find the row in the terms metadata file for the term to be modified
        for term_rownumber in range(1, len(terms_metadata)):
            if mods_localname_string == terms_metadata[term_rownumber][term_localName]:
                terms_metadata[term_rownumber][term_modified_dateTime] = isoTime(local_offset_from_utc)
                terms_metadata[term_rownumber][term_modified] = date_issued
                # replace every column that's in the modifications metadata
                for column_number in range(0, len(modifications_metadata[0])):
                    # find the column in the current terms metadata table that matches the modifications column and replace the current term's value
                    result = findColumnWithHeader(terms_metadata[0], modifications_metadata[0][column_number])
                    if result[0] == True:
                        terms_metadata[term_rownumber][result[1]] = modifications_metadata[mods_rownumber][column_number]
                    else:
                        pass # this shouldn't really happen since there already was a check that all columns existed in the versions table
                print(terms_metadata[term_rownumber])
    # this section of code adds new term metadata
    else: 
        newTermRow = []
        for column in range(0, len(terms_metadata[0])):
            newTermRow.append('')
        newTermRow[term_modified_dateTime] = isoTime(local_offset_from_utc)
        newTermRow[term_modified] = date_issued
        newTermRow[term_created] = date_issued
        newTermRow[term_isDefinedBy] = namespaceUri
        # replace every column that's in the modifications metadata
        for column_number in range(0, len(modifications_metadata[0])):
            # find the column in the current terms metadata table that matches the modifications column and replace the current term's value
            result = findColumnWithHeader(terms_metadata[0], modifications_metadata[0][column_number])
            if result[0] == True:
                newTermRow[result[1]] = modifications_metadata[mods_rownumber][column_number]
            else:
                pass # this shouldn't really happen since there already was a check that all columns existed in the versions table
        print(newTermRow)
        terms_metadata.append(newTermRow)
writeCsv('temp_terms.csv', terms_metadata)

# 5 Generate new term list version

These changes to terms trigger a new version of the term list.  The term lists are in the special directories `term-lists` and `term-lists-versions`.  

## 5.1 Open tables

The master term lists and term list versions joins tables are similar to the terms metadata tables and their joins tables:

In [ ]:
term_lists_table_filename = 'term-lists/term-lists.csv'
term_lists_table = readCsv(term_lists_table_filename)
print('Term lists table headers and first row: ', term_lists_table[0:2])

In [ ]:
term_lists_versions_joins_filename = 'term-lists/term-lists-versions.csv'
term_lists_versions_joins = readCsv(term_lists_versions_joins_filename)
print('Term lists versions joins table headers and first row: ', term_lists_versions_joins[0:2])

The `term-list-members.csv` file is used to create the one:many links between the term list and the terms that are members of it.

In [ ]:
term_lists_members_filename = 'term-lists/term-lists-members.csv'
term_lists_members = readCsv(term_lists_members_filename)
print('Term lists members table headers and first row: ', term_lists_members[0:2])

The master term list versions metadata table is used to generate records of the versions.

In [ ]:
term_lists_versions_metadata_filename = 'term-lists-versions/term-lists-versions.csv'
term_lists_versions_metadata = readCsv(term_lists_versions_metadata_filename)
print('Term lists versions metadata table headers and first row: ', term_lists_versions_metadata[0:2])

The term lists versions members table tracks all of the term versions that are part of a particular version of a term list. It's used to generate `dcterms:hasPart` and `dcterms:isPartOf` links.

In [ ]:
term_lists_versions_members_filename = 'term-lists-versions/term-lists-versions-members.csv'
term_lists_versions_members = readCsv(term_lists_versions_members_filename)
print('Term lists versions members table headers and first row: ', term_lists_versions_members[0:2])

The term lists versions replacements is used to generate the `dcterms:replaces` and `dcterms:isReplacedBy` links between term list versions.

In [ ]:
term_lists_versions_replacements_filename = 'term-lists-versions/term-lists-versions-replacements.csv'
term_lists_versions_replacements = readCsv(term_lists_versions_replacements_filename)
print('Term lists versions replacements table headers and first row: ', term_lists_versions_replacements[0:2])

## 5.2 Update tables

Generate the URI for the new term list version.
**Note:** need to trap for ideosyncratic URI pattern of `http://rs.tdwg.org/dwc/terms/attributes/`

In [ ]:
if namespaceUri == 'http://rs.tdwg.org/dwc/terms/attributes/':
    termlistVersionUri = 'http://rs.tdwg.org/dwc/version/terms/attributes/' + date_issued
else:
    # split the URI between the vocabulary and term list subpaths
    uriPieces = namespaceUri.split('/')
    termlistVersionUri = uriPieces[0] + '//' + uriPieces[2] + '/' + uriPieces[3] + '/version/' + uriPieces[4] + '/' + date_issued
    print(uriPieces)
    print(termlistVersionUri)

Update the `list_modified` value for the focal term list. **Note:** Any modifications to the term list label or description needs to be done manually to the CSV file.

In [ ]:
list_uri = findColumnWithHeader(term_lists_table[0], 'list')[1]
list_modified = findColumnWithHeader(term_lists_table[0], 'list_modified')[1]
modified_datetime = findColumnWithHeader(term_lists_table[0], 'document_modified')[1]

for rowNumber in range(1, len(term_lists_table)):
    # by convention, the namespace URI used for the terms is the same as the URI of the term list
    if namespaceUri == term_lists_table[rowNumber][list_uri]:
        term_list_rowNumber = rowNumber
        term_lists_table[rowNumber][list_modified] = date_issued
        term_lists_table[rowNumber][modified_datetime] = isoTime(local_offset_from_utc)
        print(term_lists_table[rowNumber])

writeCsv('term-lists.csv', term_lists_table)

Add a row for the new term list version in the term list versions joins file

In [ ]:
term_lists_versions_joins.append([termlistVersionUri, namespaceUri])
writeCsv('term-lists-versions-join.csv', term_lists_versions_joins)

Add only the new terms to the term list members table

In [ ]:
for newTerm in new_terms:
    term_lists_members.append([namespaceUri, namespaceUri + newTerm])
writeCsv('term-lists-members.csv', term_lists_members)

Add new term list version to the master term list version metadata file. The script assumes that all of the metadata remains the same as the previous version. If any values are different, change the CSV manually.

In [ ]:
# find the columns than contain needed information
list_uri = findColumnWithHeader(term_lists_versions_metadata[0], 'list')[1]
document_modified = findColumnWithHeader(term_lists_versions_metadata[0], 'document_modified')[1]
version_uri = findColumnWithHeader(term_lists_versions_metadata[0], 'version')[1]
version_modified = findColumnWithHeader(term_lists_versions_metadata[0], 'version_modified')[1]
status_column = findColumnWithHeader(term_lists_versions_metadata[0], 'status')[1]

# find the most recent previous version of the term list
mostRecent = 'a' # start the value of mostRecent as something earlier alphabetically than all of the list URIs
mostRecentListNumber = 0 # dummy list number to be replaced when most recent list is found
for termListRowNumber in range(1, len(term_lists_versions_metadata)):
    # the row is one of the versions of the list
    if term_lists_versions_metadata[termListRowNumber][list_uri] == namespaceUri:
        # Make the version of the row the mostRecent if it's later than the previous mostRecent
        if term_lists_versions_metadata[termListRowNumber][list_uri] > mostRecent:
            mostRecent = term_lists_versions_metadata[termListRowNumber][list_uri]
            mostRecentListNumber = termListRowNumber

# change the status of the most recent list to superceded
term_lists_versions_metadata[mostRecentListNumber][status_column] = 'superseded'
            
# start the new list row with the metadata from the most recent list
newListRow = copy.deepcopy(term_lists_versions_metadata[mostRecentListNumber])

# substitute metadata to make the most recent list have the modified dates for the new list
newListRow[document_modified] = isoTime(local_offset_from_utc)
newListRow[version_uri] = termlistVersionUri
newListRow[version_modified] = date_issued
newListRow[status_column] = 'recommended'

# append the new term list row to the old list of term lists
term_lists_versions_metadata.append(newListRow)

# save as a file
writeCsv('term-lists-versions.csv', term_lists_versions_metadata)

Create the term list version members list for the new version of the term list.  This includes every pre-existing term version that hasn't changed, the new versions of terms that changed, and term versions for new terms that weren't previously on the list.

As the script updates term list version members, it will also create a new row in the term list replacements table if the new term version is replacing a previous one.

In [ ]:
# create a list of every term version that was in the most recent previous list version
newTermVersionMembersList = []
# create a corresponding list of local names for those versions
termLocalNameList = []

for termVersion in term_lists_versions_members:
    # the first column contains the term list version
    if term_lists_versions_metadata[mostRecentListNumber][version_uri] == termVersion[0]:
        newTermVersionMembersList.append(termVersion[1])
        
        # dissect the term version URI to pull out the local name of the term version
        pieces = termVersion[1].split('/')
        versionLocalNamePiece = pieces[len(pieces)-1]
        # split off the local name string from the issue date part of the version local name
        termLocalNameList.append(versionLocalNamePiece.split('-')[0])

# For each modified term, find its previous version and replace it with the new new version.
for modified_term in modified_terms:
    for termVersionRowNumber in range(0, len(newTermVersionMembersList)):
        if modified_term == termLocalNameList[termVersionRowNumber]:
            # change the version on the list to the new one
            newTermVersionMembersList[termVersionRowNumber] = namespaceUri + 'version/' + termLocalNameList[termVersionRowNumber] + '-' + date_issued

# For each newly added term, add its new version to the list.
for new_term in new_terms:
    newTermVersionMembersList.append(namespaceUri + 'version/' + new_term + '-' + date_issued)

# Now that the list is created of new term versions that are part of the new term version list,
# add a record for each one to the term list versions members table
for termVersionMember in newTermVersionMembersList:
    term_lists_versions_members.append([termlistVersionUri, termVersionMember])

# Write the updated term list versions members table to a file
writeCsv('term-lists-versions-members.csv', term_lists_versions_members)

Add a record to the term list versions replacements table showing that the new term list version has replaced the previous one.

In [ ]:
term_lists_versions_replacements.append([termlistVersionUri, term_lists_versions_metadata[mostRecentListNumber][version_uri]])
writeCsv('term-lists-versions-replacements.csv', term_lists_versions_replacements)

# 6 Generate new vocabulary version

**Note:** This assumes that there will only be a single term list that has been modified in a particular vocabulary update.  If this script is run multiple times to update more than one term list within the vocabulary, ...

The update process is very similar to that of the term lists.

## 6.1 Open tables

The types of tables related to vocabularies and their versions are very similar to those of term lists.  The are located in the directories `vocabularies` and `vocabularies-versions`. 

In [ ]:
vocabularies_table_filename = 'vocabularies/vocabularies.csv'
vocabularies_table = readCsv(vocabularies_table_filename)
print('Vocabularies table headers and first row: ', vocabularies_table[0:2])

In [ ]:
vocabularies_versions_joins_filename = 'vocabularies/vocabularies-versions.csv'
vocabularies_versions_joins = readCsv(vocabularies_versions_joins_filename)
print('Vocabularies versions joins table headers and first row: ', vocabularies_versions_joins[0:2])

In [ ]:
vocabularies_members_filename = 'vocabularies/vocabularies-members.csv'
vocabularies_members = readCsv(vocabularies_members_filename)
print('Vocabularies members table headers and first row: ', vocabularies_members[0:2])

In [ ]:
vocabularies_versions_metadata_filename = 'vocabularies-versions/vocabularies-versions.csv'
vocabularies_versions_metadata = readCsv(vocabularies_versions_metadata_filename)
print('Vocabularies versions metadata table headers and first row: ', vocabularies_versions_metadata[0:2])

In [ ]:
vocabularies_members_filename = 'vocabularies-versions/vocabularies-versions-members.csv'
vocabularies_members = readCsv(vocabularies_members_filename)
print('Vocabularies versions members table headers and first row: ', vocabularies_members[0:2])

In [ ]:
vocabularies_versions_replacements_filename = 'vocabularies-versions/vocabularies-versions-replacements.csv'
vocabularies_versions_replacements = readCsv(vocabularies_versions_replacements_filename)
print('Vocabularies versions replacements table headers and first row: ', vocabularies_versions_replacements[0:2])

## 6.2 Update tables

**Note:** The steps here are analogous to updating term lists.

Generate the URI for the new vocabulary version

In [ ]:
# find the vocabulary subpath for the updated term list
list_localName_column = findColumnWithHeader(term_lists_table[0], 'list_localName')[1]
list_localName = term_lists_table[term_list_rowNumber][list_localName_column]
print('List local name: ', list_localName)
# the vocabulary subpath is the first part of the list local name
vocab_subpath = list_localName.split('/')[0]
print('Vocabulary subpath: ', vocab_subpath)

# generate the vocabulary version URI
vocabularyVersionUri = 'http://rs.tdwg.org/version/' + vocab_subpath + '/' + date_issued
print('New version URI: ', vocabularyVersionUri)
